# Load libraries

In [519]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import fiona


# Import common GIS tools
import numpy as np

import matplotlib.pyplot as plt
import rasterio.mask
from matplotlib.cm import jet,RdYlGn

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer 
from odc.stac import stac_load

# Not sure that I would use
import folium


# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

In [520]:
import osmnx as ox
from shapely.geometry import Point, Polygon


# Load training data


In [521]:
ground_df = pd.read_csv("Training_data_uhi_index_2025-02-18.csv")
print(ground_df.shape)
ground_df.head()

(11229, 4)


,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


In [522]:
#Date time convert
ground_df['datetime'] = pd.to_datetime(ground_df['datetime'])


In [523]:
place_name = "New York City, USA"
counties = ox.features_from_place(place_name, tags={"boundary": "administrative", "admin_level": "7"})

borough_names = ["Manhattan", "The Bronx"]
counties = counties[counties['name'].isin(borough_names)]


ground_df['geometry'] = ground_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf_points = gpd.GeoDataFrame(ground_df, geometry='geometry', crs=counties.crs)

if "index_right" in gdf_points.columns:
    gdf_points.drop(columns=["index_right"], inplace=True)
# Join
ground_df= gpd.sjoin(gdf_points,counties, how='left', predicate='within')

# Longitude/Latitude/datetime/UHI Index
ground_df = ground_df[['Longitude','Latitude','datetime','UHI Index','geometry','name']]
ground_df.rename(columns={'name':"Counties"}, inplace=True)

In [524]:
ground_df


,Longitude,Latitude,datetime,UHI Index,geometry,Counties
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,POINT (-73.90917 40.81311),The Bronx
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,POINT (-73.90919 40.81304),The Bronx
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,POINT (-73.90922 40.81298),The Bronx
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,POINT (-73.90924 40.81291),The Bronx
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,POINT (-73.90926 40.81284),The Bronx
...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,POINT (-73.95705 40.79033),Manhattan
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,POINT (-73.95706 40.79031),Manhattan
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,POINT (-73.95709 40.79027),Manhattan
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,POINT (-73.95711 40.79025),Manhattan


In [525]:
ground_df["Counties"].value_counts()

Counties
Manhattan    5819
The Bronx    5410
Name: count, dtype: int64

# Weather data

- 1- Mapping the weather for Manhattan and Bronx from additional data

### Bronx and Manhattan

In [526]:
dataset_bronx = pd.read_excel("Additional dataset/NY_Mesonet_Weather.xlsx", sheet_name= "Bronx")
dataset_manhattan = pd.read_excel("Additional dataset/NY_Mesonet_Weather.xlsx", sheet_name="Manhattan")

In [527]:

#change datetime of weather bronx and weather manhattan
dataset_bronx['Date / Time'] = pd.to_datetime(dataset_bronx['Date / Time'])
dataset_manhattan['Date / Time'] = pd.to_datetime(dataset_manhattan['Date / Time'])
dataset_bronx = dataset_bronx.sort_values('Date / Time')
dataset_manhattan = dataset_manhattan.sort_values('Date / Time')

In [528]:
dataset_bronx

,Date / Time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,2021-07-24 06:00:00,19.3,88.2,0.8,335,12
1,2021-07-24 06:05:00,19.4,87.9,0.8,329,18
2,2021-07-24 06:10:00,19.3,87.6,0.7,321,25
3,2021-07-24 06:15:00,19.4,87.4,0.5,307,33
4,2021-07-24 06:20:00,19.4,87.0,0.2,301,42
...,...,...,...,...,...,...
164,2021-07-24 19:40:00,24.9,49.0,3.5,184,24
165,2021-07-24 19:45:00,24.8,49.0,3.3,173,19
166,2021-07-24 19:50:00,24.9,48.7,3.8,168,17
167,2021-07-24 19:55:00,24.9,47.3,4.1,171,16


#### Merge data

In [529]:
# Sort dataframes by datetime for asof merge
ground_df = ground_df.sort_values('datetime')

# Merge Bronx data
bronx_data = pd.merge_asof(
    ground_df[ground_df['Counties'] == 'Bronx'],
    dataset_bronx,
    left_on='datetime',
    right_on='Date / Time',
    direction='nearest'  # Matches nearest timestamp
)

# Merge Manhattan data
manhattan_data = pd.merge_asof(
    ground_df[ground_df['Counties'] == 'Manhattan'],
    dataset_manhattan,
    left_on='datetime',
    right_on='Date / Time',
    direction='nearest'
)

# Combine the two datasets back into one
final_df = pd.concat([bronx_data, manhattan_data])

# Drop redundant 'Date / Time' column if necessary
final_df.drop(columns=['Date / Time'], inplace=True)

# Reset index
final_df.reset_index(drop=True, inplace=True)

print(len(final_df))
final_df

5819


,Longitude,Latitude,datetime,UHI Index,geometry,Counties,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.977157,40.771555,2021-07-24 15:01:00,0.975234,POINT (-73.97716 40.77156),Manhattan,26.1,51.1,4.1,139,140
1,-73.978742,40.771675,2021-07-24 15:01:00,0.975234,POINT (-73.97874 40.77168),Manhattan,26.1,51.1,4.1,139,140
2,-73.978453,40.771677,2021-07-24 15:01:00,0.970907,POINT (-73.97845 40.77168),Manhattan,26.1,51.1,4.1,139,140
3,-73.978338,40.771697,2021-07-24 15:01:00,0.973071,POINT (-73.97834 40.7717),Manhattan,26.1,51.1,4.1,139,140
4,-73.978227,40.771717,2021-07-24 15:01:00,0.973071,POINT (-73.97823 40.77172),Manhattan,26.1,51.1,4.1,139,140
...,...,...,...,...,...,...,...,...,...,...,...
5814,-73.979088,40.771655,2021-07-24 15:59:00,0.975234,POINT (-73.97909 40.77166),Manhattan,27.0,46.1,2.7,209,620
5815,-73.979060,40.771682,2021-07-24 15:59:00,0.975234,POINT (-73.97906 40.77168),Manhattan,27.0,46.1,2.7,209,620
5816,-73.979027,40.771695,2021-07-24 15:59:00,0.975234,POINT (-73.97903 40.7717),Manhattan,27.0,46.1,2.7,209,620
5817,-73.978947,40.771705,2021-07-24 15:59:00,0.975234,POINT (-73.97895 40.7717),Manhattan,27.0,46.1,2.7,209,620


# Footprint data

In [530]:
buildings = gpd.read_file('Additional dataset/Building_Footprint.kml', driver='KML')
buildings

,Name,Description,geometry
0,,,"MULTIPOLYGON (((-73.91903 40.8482, -73.91933 4..."
1,,,"MULTIPOLYGON (((-73.92195 40.84963, -73.92191 ..."
2,,,"MULTIPOLYGON (((-73.9205 40.85011, -73.92045 4..."
3,,,"MULTIPOLYGON (((-73.92056 40.8514, -73.92053 4..."
4,,,"MULTIPOLYGON (((-73.91234 40.85218, -73.91247 ..."
...,...,...,...
9431,,,"MULTIPOLYGON (((-73.95267 40.77923, -73.95254 ..."
9432,,,"MULTIPOLYGON (((-73.94964 40.77613, -73.94931 ..."
9433,,,"MULTIPOLYGON (((-73.9521 40.7688, -73.95174 40..."
9434,,,"MULTIPOLYGON (((-73.9523 40.75904, -73.95246 4..."


In [531]:
buildings = buildings.explode(column="geometry")
buildings

,Name,Description,geometry
0,,,"POLYGON ((-73.91903 40.8482, -73.91933 40.8479..."
1,,,"POLYGON ((-73.92195 40.84963, -73.92191 40.849..."
2,,,"POLYGON ((-73.9205 40.85011, -73.92045 40.8501..."
3,,,"POLYGON ((-73.92056 40.8514, -73.92053 40.8514..."
4,,,"POLYGON ((-73.91234 40.85218, -73.91247 40.852..."
...,...,...,...
9431,,,"POLYGON ((-73.95267 40.77923, -73.95254 40.779..."
9432,,,"POLYGON ((-73.94964 40.77613, -73.94931 40.776..."
9433,,,"POLYGON ((-73.9521 40.7688, -73.95174 40.76931..."
9434,,,"POLYGON ((-73.9523 40.75904, -73.95246 40.7590..."


In [532]:
# Project the data to a suitable CRS for accurate area calculations
crs = "EPSG:32618"  # Example: UTM zone 18N for New York area, adjust based on your location
buildings = buildings.to_crs(crs)

# Calculate the area of each polygon
buildings['area'] = buildings.geometry.area

# Calculate the total area of all buildings
total_area = buildings['area'].sum()

# Calculate the percentage of the total area that each building occupies
buildings['percentage_of_total_area'] = (buildings['area'] / total_area) * 100

In [533]:
buildings

,Name,Description,geometry,area,percentage_of_total_area
0,,,"POLYGON ((591120.366 4522468.172, 591095.414 4...",619.739128,0.003288
1,,,"POLYGON ((590872.014 4522623.878, 590875.401 4...",95.264377,0.000505
2,,,"POLYGON ((590993.579 4522679.112, 590997.635 4...",141.262523,0.000749
3,,,"POLYGON ((590986.925 4522822.254, 590989.312 4...",79.661011,0.000423
4,,,"POLYGON ((591678.478 4522917.522, 591667.723 4...",216.099170,0.001146
...,...,...,...,...,...
9431,,,"POLYGON ((588376.201 4514777.789, 588386.742 4...",2487.752472,0.013197
9432,,,"POLYGON ((588636.003 4514436.833, 588662.794 4...",9055.028398,0.048035
9433,,,"POLYGON ((588437.793 4513620.547, 588467.841 4...",9523.114836,0.050518
9434,,,"POLYGON ((588434.276 4512536.693, 588420.207 4...",1545.474820,0.008198


In [543]:
# Step 1: Ensure both GeoDataFrames have the same CRS
if final_df.crs != buildings.crs:
    buildings = buildings.to_crs(final_df.crs)  # Reproject buildings to match final_df's CRS

# Step 3: Perform a spatial join to map points to polygons
# Use 'within' to find points that fall inside polygons
mapped_data = gpd.sjoin(final_df, buildings[['geometry', 'area', 'percentage_of_total_area']], how="left", predicate="within")

# Step 4: Drop the index_right column created by the spatial join (optional)
mapped_data = mapped_data.drop(columns=['index_right'])

# Step 5: Display the resulting DataFrame
mapped_data

,Longitude,Latitude,datetime,UHI Index,geometry,Counties,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],area,percentage_of_total_area
0,-73.977157,40.771555,2021-07-24 15:01:00,0.975234,POINT (-73.97716 40.77156),Manhattan,26.1,51.1,4.1,139,140,NaN,NaN
1,-73.978742,40.771675,2021-07-24 15:01:00,0.975234,POINT (-73.97874 40.77168),Manhattan,26.1,51.1,4.1,139,140,NaN,NaN
2,-73.978453,40.771677,2021-07-24 15:01:00,0.970907,POINT (-73.97845 40.77168),Manhattan,26.1,51.1,4.1,139,140,NaN,NaN
3,-73.978338,40.771697,2021-07-24 15:01:00,0.973071,POINT (-73.97834 40.7717),Manhattan,26.1,51.1,4.1,139,140,NaN,NaN
4,-73.978227,40.771717,2021-07-24 15:01:00,0.973071,POINT (-73.97823 40.77172),Manhattan,26.1,51.1,4.1,139,140,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5814,-73.979088,40.771655,2021-07-24 15:59:00,0.975234,POINT (-73.97909 40.77166),Manhattan,27.0,46.1,2.7,209,620,NaN,NaN
5815,-73.979060,40.771682,2021-07-24 15:59:00,0.975234,POINT (-73.97906 40.77168),Manhattan,27.0,46.1,2.7,209,620,NaN,NaN
5816,-73.979027,40.771695,2021-07-24 15:59:00,0.975234,POINT (-73.97903 40.7717),Manhattan,27.0,46.1,2.7,209,620,NaN,NaN
5817,-73.978947,40.771705,2021-07-24 15:59:00,0.975234,POINT (-73.97895 40.7717),Manhattan,27.0,46.1,2.7,209,620,NaN,NaN


In [544]:
mapped_data = mapped_data.drop_duplicates(subset=['Longitude', 'Latitude', 'datetime'])

In [546]:
mapped_data[mapped_data['area'].notna()].head()

,Longitude,Latitude,datetime,UHI Index,geometry,Counties,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],area,percentage_of_total_area
276,-73.972632,40.764532,2021-07-24 15:08:00,1.012498,POINT (-73.97263 40.76453),Manhattan,26.3,50.9,3.0,158,219,7881.149254,0.041808
277,-73.972588,40.764522,2021-07-24 15:08:00,1.012498,POINT (-73.97259 40.76452),Manhattan,26.3,50.9,3.0,158,219,7881.149254,0.041808
278,-73.972542,40.764508,2021-07-24 15:08:00,1.010335,POINT (-73.97254 40.76451),Manhattan,26.3,50.9,3.0,158,219,7881.149254,0.041808
279,-73.972497,40.764492,2021-07-24 15:08:00,1.010335,POINT (-73.9725 40.76449),Manhattan,26.3,50.9,3.0,158,219,7881.149254,0.041808
280,-73.972450,40.764472,2021-07-24 15:08:00,1.012498,POINT (-73.97245 40.76447),Manhattan,26.3,50.9,3.0,158,219,7881.149254,0.041808


# Landsat 8 data


In [535]:
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):
    
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs
    
    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)
    
    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B04_values = []
    B03_values =[]
    B02_values =[]
    B05_values=[]
    swir1_values=[]
    lwir11_values=[]


# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)
    
    # #red
    #     B04_value = data.sel(x=lon, y=lat,  band=2, method="nearest").values
    #     B04_values.append(B04_value)
    # #green
    #     B03_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
    #     B03_values.append(B03_value)

    # #blue    
    #     B02_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
    #     B02_values.append(B02_value)
    
    # #nir08 band 5
    #     B05_value = data.sel(x=lon, y=lat, band=5, method="nearest").values
    #     B05_values.append(B05_value)
    
    # # swir16
    #     swir1_value = data.sel(x=lon, y=lat, band=6, method="nearest").values
    #     swir1_values.append(swir1_value)
    # Land Surface Temperature
        lwir11_value = data.sel(x=lon, y=lat, band=1, method="nearest").values
        lwir11_values.append(lwir11_value)


    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    # df['B04_LS'] = B04_values
    # df['B03_LS'] = B03_values
    # df['B02_LS'] = B02_values
    # df['B05_LS'] = B05_values
    # df['SWIR1_LS'] = swir1_values
    df['LST'] = lwir11_values
    
    return df


In [536]:
# Mapping satellite data with training data.
final_data = map_satellite_data('Landsat_LST.tiff', 'Training_data_uhi_index_2025-02-18.csv')

Mapping values: 100%|██████████| 11229/11229 [00:20<00:00, 557.14it/s]


In [537]:
# #ndiv
# final_data['NDVI_LS'] = (final_data['B05_LS'] - final_data['B04_LS']) / (final_data['B05_LS'] + final_data['B04_LS'])
# final_data['NDVI_LS'] = final_data['NDVI_LS'].replace([np.inf, -np.inf], np.nan)

In [538]:
# #ndbi
# final_data['NDBI_LS'] = (final_data['SWIR1_LS'] - final_data['B05_LS']) / (final_data['SWIR1_LS'] + final_data['B05_LS'])
# final_data['NDBI_LS'] = final_data['NDBI_LS'].replace([np.inf, -np.inf], np.nan)

In [539]:
# ##NDWI
# final_data['NDWI_LS'] = (final_data['B03_LS'] - final_data['B05_LS']) / (final_data['B03_LS'] + final_data['B05_LS'])
# final_data['NDWI_LS'] = final_data['NDVI_LS'].replace([np.inf, -np.inf], np.nan)

In [540]:
final_data.head()

,LST
0,37.78553354000002
1,37.78553354000002
2,37.78553354000002
3,37.35828104000001
4,37.35828104000001


In [541]:
final_data.describe()

,LST
count,11229
unique,1991
top,36.98229884
freq,32


# Sentinel-2 data